In [ ]:
! pip install kmc2

In [ ]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

In [ ]:
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import DictionaryLearning
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparseCoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from imutils import paths

import matplotlib.pyplot as plt
import scipy.cluster.vq as vq
import _pickle as cPickle
import pandas as pd
import numpy as np
import imutils 
import torch
import time
import math
import copy
import kmc2
import cv2 
import os

## 데이터 불러오기

In [ ]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle -v

In [ ]:
! kaggle competitions download -c 2019-ml-finalproject
! A | unzip 2019-ml-finalproject.zip

In [ ]:
caltech_dir = "/content/train/"
lists = paths.list_images(caltech_dir)

In [ ]:
X = []
y = []

for l in lists:
  path = os.path.dirname(l)
  # if l[15] != 'B':
  img = cv2.imread(l)
  img = cv2.resize(img, dsize=(256, 256))
  X.append(img)
  y.append(os.path.basename(path))

# categories.remove('BACKGROUND_Google') # it contains all kinds of images. 
print(len(X))

In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(img_gray, None)
        x.append(des)

    return x

In [ ]:
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        step_size = 8
        kp = [cv2.KeyPoint(x, y, step_size)
              for x in range(0, img_gray.shape[0], step_size) 
                for y in range(0, img_gray.shape[1], step_size)]
        dense_feat = sift.compute(img_gray, kp)
        x.append(dense_feat[1])
        
    return x

In [ ]:
X_sift = computeSIFT(X)

In [ ]:
X_sift = np.array(X_sift)
X_sift.shape

In [ ]:
all_train_desc = np.vstack((descriptor for descriptor in X_sift))

In [ ]:
all_train_desc.shape

In [ ]:
# with open('/content/drive/My Drive/Colab Notebooks/all_train_desc_sc.pkl','wb') as f:
#   cPickle.dump(all_train_desc, f)

In [ ]:
def clusterFeatures(all_train_desc, k):
    seeding = kmc2.kmc2(all_train_desc, k)
    model = MiniBatchKMeans(k, init=seeding).fit(all_train_desc)
    return model

In [ ]:
kmeans = clusterFeatures(all_train_desc, 600)

In [ ]:
def extract_denseSIFT(img):
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = 8
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img_gray.shape[0], disft_step_size)
                for x in range(0, img_gray.shape[1], disft_step_size)]

    descriptors = sift.compute(img_gray, keypoints)[1]

    return descriptors

In [ ]:
def getImageFeaturesSPM(L, img, kmeans, k):
    W = 256
    H = 256
    step_size=8
    W_des = int(W/step_size)
    H_des = int(H/step_size)  
    h = []

    img = img.reshape(H_des, W_des, 128)
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):
                desc = img[y:y+h_step, x:x+w_step, :].reshape(-1, 128)
                histo = input_vector_encoder(desc, kmeans.cluster_centers_)
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    if L == 0:
        hist = hist.astype('float64')
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist

In [ ]:
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [ ]:
def input_vector_encoder(feature, codebook):
    code, _ = vq(feature, codebook)
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
    return word_hist

In [ ]:
train_histo = getHistogramSPM(2, X_sift, kmeans, 600)

In [ ]:
clf = LinearSVC(random_state=0, C=0.001)
clf.fit(train_histo, y)

In [ ]:
# C_range = 10.0 ** np.arange(-3, 3)
# param_grid = dict(C=C_range.tolist())
# clf = GridSearchCV(LinearSVC(), param_grid, cv=5, n_jobs=-2)
# clf.fit(train_histo, y)

In [ ]:
# clf.best_params_

In [ ]:
caltech_dir = "/content/testAll_v2/"
lists = paths.list_images(caltech_dir)

X_final = []
img_name = []
for l in lists:
  path = os.path.dirname(l)
  img = cv2.imread(l)
  img = cv2.resize(img, dsize=(256, 256))
  X_final.append(img)
  img_name.append(l[-14:])

In [ ]:
X_final = np.array(X_final)

In [ ]:
X_final.shape

In [ ]:
X_histo = getHistogramSPM(2, X_final, kmeans, 600)

In [ ]:
y_predict = clf.predict(X_histo)

In [ ]:
len(y_predict)

In [ ]:
df = pd.read_csv('/content/Label2Names.csv')

In [ ]:
df = np.array(df)

In [ ]:
result = []

In [ ]:
len(df)

In [ ]:
for i in range(len(y_predict)):
  flag = 0
  if y_predict[i] == 'BACKGROUND_Google':
    flag = 1
    result.append(101)
  else:
    for j in range(len(df)):
        if y_predict[i] == df[j][1]:
          result.append(df[j][0])
          flag = 1
  
  if flag != 1:
    result.append(1)

In [ ]:
len(result)

In [ ]:
result = np.array(result)

In [ ]:
img_name = np.array(img_name)

In [ ]:
result = result.reshape(-1, 1)
img_name = img_name.reshape(-1, 1)
total_result = np.hstack([img_name, result])
print(total_result)

In [ ]:
df = pd.DataFrame(total_result, columns=['Id', 'Category'])
df.to_csv('results-mjkim-v4.csv', index=False, header=True)

In [ ]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-mjkim-v4.csv -m "mjkim-20191123"

In [ ]:
def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
            
    return result
# 히스토그램 유사도를 측정하는 함수(위에 링크에서 가져옴) (SPM 커널 적용하는부분)

gramMatrix = histogramIntersection(train_histo, train_histo)
# SPM을 통해서 만든 feature의 유사도를 구해서 gramMatrix를 정의
clf = SVC(kernel='precomputed')
# SVC 진행시 precomputed를 통해서 위에서 정의한 SPM 커널을 기준으로 SVC 진행
clf.fit(gramMatrix, y)
# SVC 진행시 preconputed를 통해서 위에서 정의한 SPM 커널을 기준으로 SVC 진행
predictMatrix = histogramIntersection(X_histo, train_histo)
# test셋과 train셋에 대한 유사도를 구한 predictMatrix를 구함
SVMResults = clf.predict(predictMatrix)
# 이를통해서 최종 결과인 SVMResults를 구함(결과값 클래스 라벨)

In [ ]:
df = pd.read_csv('/content/Label2Names.csv')

In [ ]:
df = np.array(df)

In [ ]:
len(df)

In [ ]:
SVMResults

In [ ]:
result = []
for i in range(len(SVMResults)):
  flag = 0
  if SVMResults[i] == 'BACKGROUND_Google':
    flag = 1
    result.append(101)
  else:
    for j in range(len(df)):
        if SVMResults[i] == df[j][1]:
          result.append(df[j][0])
          flag = 1
  
  if flag != 1:
    result.append(1)

In [ ]:
result = np.array(result)

In [ ]:
result = result.reshape(-1, 1)
img_name = img_name.reshape(-1, 1)
total_result = np.hstack([img_name, result])
print(total_result)

In [ ]:
df = pd.DataFrame(total_result, columns=['Id', 'Category'])
df.to_csv('results-mjkim-v5.csv', index=False, header=True)

In [ ]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-mjkim-v5.csv -m "mjkim-20191123"

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_histo = scaler.fit_transform(train_histo)
X_histo = scaler.transform(X_histo)